# 从零开始学Python网络爬虫

In [1]:
## format（）， 格式化函数
# 在 str.format() 调用时使用关键字参数，可以通过参数名来引用值
print('This {food} is {adjective}.'.format(food='spam', adjective='absolutely horrible'))

print("{:.2f}".format(3.1415926))

"{} {}".format("hello", "world")    # 不设置指定位置，按默认顺序

path = 'https://{}/{}'.format("bd", "com")
print(path)

url = ['http://abc/p{}/'.format(number) for number in range(1,10)]
print(url)

This spam is absolutely horrible.
3.14
https://bd/com
['http://abc/p1/', 'http://abc/p2/', 'http://abc/p3/', 'http://abc/p4/', 'http://abc/p5/', 'http://abc/p6/', 'http://abc/p7/', 'http://abc/p8/', 'http://abc/p9/']


### 爬虫原理
网络连接需要一次Requests请求和服务器端的Response回应。爬虫原理：  
- 模拟电脑对服务器发起Requests请求
- 接收服务器端的Response的内容并解析、提取所需信息  

常用的两种爬虫的流程：多页面和跨页面爬虫流程。
![](./note/flow.png)

### 爬虫三大库
- **Requests**  
Requests库的错误和异常：
 - ConnectionError：网络连接错误异常，如DNS查询失败、拒绝连接等
 - HTTPError：HTTP错误异常，比如网页不存在，返回404
 - URLRequired：URL缺失异常
 - TooManyRedirects：超过最大重定向次数，产生重定向异常
 - ConnectTimeout：连接远程服务器超时异常
 - Timeout：请求URL超时，产生超时异常
- **BeautifulSoup**  
解析器： html.parser、lxml等，用法: BeautifulSoup(url.text, "html.parser"),BeautifulSoup(url.text, "lxml")  
**在浏览器中可以得到BeautifulSoup的select方法的路径： 右键 > Copy Selector**。
- **Lxml**

In [2]:
## requests
import requests

# 加入请求头，伪装成浏览器
headers = {
    'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/53.0.2785.143 Safari/537.36'
}

def get_links(url):
    wb_data = requests.get(url,headers=headers)
    print(wb_data.status_code)
    try:
        print(wb_data)
        # print(wb_data.text)
    except ConnectonError:
        print('Requests Error')

# test    
url = "http://www.baidu.com"
get_links(url)

200
<Response [200]>


In [3]:
## BeautifulSoup， select方法
from bs4 import BeautifulSoup
import requests
headers = {
    'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/53.0.2785.143 Safari/537.36'
}
url = "http://bj.xiaozhu.com/"
res = requests.get(url, headers=headers)
soup = BeautifulSoup(res.text, 'html.parser')
# 选择房价的元素的路径（右键 > Copy Selector），得到房价值
prices = soup.select('#page_list > ul > li > div.result_btm_con.lodgeunitname > div > span > i')
for price in prices:
    print(price, "\t", price.get_text())

<i>488</i> 	 488
<i>498</i> 	 498
<i>518</i> 	 518
<i>340</i> 	 340
<i>498</i> 	 498
<i>388</i> 	 388
<i>388</i> 	 388
<i>458</i> 	 458
<i>278</i> 	 278
<i>430</i> 	 430
<i>369</i> 	 369
<i>498</i> 	 498
<i>528</i> 	 528
<i>300</i> 	 300
<i>408</i> 	 408
<i>598</i> 	 598
<i>339</i> 	 339
<i>278</i> 	 278
<i>298</i> 	 298
<i>428</i> 	 428
<i>278</i> 	 278
<i>418</i> 	 418
<i>609</i> 	 609
<i>498</i> 	 498


In [4]:
## BeautifulSoup
# 需要的url如下， 
# <a target="_blank" href="http://bj.xiaozhu.com/fangzi/29968007503.html" class="resule_img_a">
# Element的Selector： page_list > ul > li:nth-child(1) > a
url = "http://bj.xiaozhu.com/search-duanzufang-p2-0/"
res = requests.get(url, headers=headers)
soup = BeautifulSoup(res.text, "lxml")
link = soup.select('#page_list > ul > li > a')
## 用相同的select方法，得到了该级元素的内容，即<a...</a>
#+ 然后用get(element_name)方法，获得"href"属性值
print(link[0], 2*"\n", link[0].get("href"))

## 同时，我们可以进一步用相同的方法提取： title、img等信息

<a class="resule_img_a" href="http://bj.xiaozhu.com/fangzi/10573777860.html" target="_blank">
<img alt="近国贸百子湾地铁-经典黑白loft-可拍摄" class="lodgeunitpic" data-growing-title="10573777860" lazy_src="https://image.xiaozhustatic3.com/12/10,0,43,4243,1800,1201,c8e51b89.jpg" src="../images/lazy_loadimage.png" title="近国贸百子湾地铁-经典黑白loft-可拍摄"/>
</a> 

 http://bj.xiaozhu.com/fangzi/10573777860.html


### 实践Task： 爬取酷狗Top500的数据
**方法是： requests+BeautifulSoup**
- url： https://www.kugou.com/yy/rank/home/1-8888.html?from=rank
- 代码： [kugou.py](./book_src/kugou.py),用Python3运行 （修复了原代码书中的一个bug）
- 思路： (1)观察翻页的各页url主入口如何获取； (2)分别在各页爬取

### 正则表达式： Python re模块
- search()
- sub()
- findall()  

可以用正则表达式直接解析返回的html文件，得到有用的信息。

In [5]:
import re
## re.search()
a = "one1two2three3"
info = re.search('\D+', a)
print(info, "\n", info.group(), "\n")

## re.sub()
new_info = re.sub('\d+', ' ', a)
print(new_info)

## re.findall()
infos = re.findall('\D+', a)
print("\n", infos)

a= "<a>123</a><a>456</a><a>789</a>"
## 边界匹配，括号里的内容作为返回结果
infos = re.findall('<a>(.*?)</a>', a)
print("\n", infos)


<_sre.SRE_Match object; span=(0, 3), match='one'> 
 one 

one two three 

 ['one', 'two', 'three']

 ['123', '456', '789']


### 实践Task： 爬取《斗破苍穹》全文小说
**方法是： requests+re**
- url: http://www.doupoxs.com/doupocangqiong/
- 代码： [doupo_xiaoshuo.py](./book_src/doupo_xiaoshuo.py),用Python3运行
- `content.decode('utf-8')`
- `re.S`， re修饰符， 匹配包含换行在内的所有字符
- `time.sleep(1)`， 防止请求频率过快导致爬虫失败  
注： **最好加入一个try/except判断，如果请求因过快被拒绝，则重新连接， 保证数据的完整性。**

### 实践Task： 爬取糗事百科的段子
**方法是： requests+re**
- url: https://www.qiushibaike.com/text/
- 代码： [qiushibaike.py](./book_src/qiushibaike.py),用Python3运行
- P.S. 对段子中的`"</br>"`字符串，需要替换删除

### Lxml库 + Xpath语法
lxml库是用来解析XML和HTML文件的一个Python库。  
Xpath是一门在XML文档中查找信息的语言，同时支持HTML文档。  
可参考： [Xpath-菜鸟教程](https://www.runoob.com/xpath/xpath-tutorial.html)  
**在爬虫实战中，Xpath路径可以通过浏览器得到： 右键 > Copy Xpath。**  

|表达式|描述/结果|
|-|-|
|/|从根节点选取|
|//|从匹配的当前节点选取|
|..|选取当前节点的父节点|
|@|选取属性|
|-|-|
|/cnode|选取根元素cnode|
|cnode/node|选取属于cnode的子元素的所有node元素|
|//cnode|选取所有cnode子元素，不管它们在文档中的位置|
|cnode//node|选取属于cnode的所有node子元素，不管它们位于cnode之下的什么位置|
|//@attribute|选取名为attribute的所有属性|
|//li[@attr]|选取所有拥有名为attr属性的li元素|
|//li[@attr="red"]|选取所有attr属性值为red的li元素|
|/text()|获取标签中的文本|

#### 比较三种方法：re/BeautifulSoup/Lxml
- 代码： [compare.py](./book_src/compare.py)
- 速度： Lxml ≈ 正则 > BeautifulSoup
- 难度： Lxml ≈ BeautifulSoup < 正则  

所以当爬取的数据量大，且需要快速实现时，**选择Lxml是最佳选择，因为速度快，实现简单。**

In [6]:
## Xpath， 获取糗事百科/文字中的一个页面下的所有user name
import requests
# lxml库
from lxml import etree
headers = {
    'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/53.0.2785.143 Safari/537.36'
}
url = 'http://www.qiushibaike.com/text/'
res = requests.get(url,headers=headers)
## etree.HTML
selector = etree.HTML(res.text)
usernames_path = selector.xpath('//div[@class="article block untagged mb15 typs_long" or  \
                                @class="article block untagged mb15 typs_hot" or  \
                                @class="article block untagged mb15 typs_old"]')
## 以上复杂的形式，可以用 starts-with 简化、替代
usernames_path = selector.xpath('//div[starts-with(@class, "article block untagged mb15")]')

for username_path in usernames_path:
    if username_path.xpath('div[1]/a[2]/h2'):
        ## 因为返回的是list，所以取唯一的一个元素即可， [0]
        username = username_path.xpath('div[1]/a[2]/h2/text()')[0].strip()
        print(username)
    else:  # 匿名用户没有以上节点
        pass



你好i八月
秀的拽，你不配
狸土豆大人
李中东
琉之璃
不喜欢洗锅
鹰嫂
（我叫红包）
吃个榴莲压压惊
没有你生而何欢？
偷惢
隔壁ㄨ老王
小呆妹！
-小门神~
youMyWay
猪猪小二
老巫婆～～
内涵葮子TV、
夕冬温存
她要它爱他
偷惢
鱼歌浅唱
-小门神~
又一盏素酒
查查猪吃


#### 用 string(.) 获取标签套标签的文本内容
[文言文-古诗文网](https://www.gushiwen.org/shiwen/default_4A444444444444A1.aspx)中的多段长文和只有一段的文言文的节点设置不同。  
此时，为了把div节点下的所有文本都获取，可以用 string(.)解决该问题。    
![](./note/stringall.png)  
![](./note/stringall2.png) 

In [7]:
## 函数， 代码复用
def stringall(url, name):
    res = requests.get(url, headers=headers)
    selector = etree.HTML(res.text)
    guwen_path = selector.xpath('//div[@class="contson"]')[0]
    ## string(.)
    guwen = guwen_path.xpath('string(.)').strip()
    info = "******" + name + "******"
    print(info)
    print(guwen)
    print()
    
## 桃花源记
url = "https://so.gushiwen.org/shiwenv_73add8822103.aspx"
stringall(url, "桃花源记")

## 陋室铭
url = "https://so.gushiwen.org/shiwenv_6c1ea9b7dd44.aspx"
stringall(url, "陋室铭")

******桃花源记******
晋太元中，武陵人捕鱼为业。缘溪行，忘路之远近。忽逢桃花林，夹岸数百步，中无杂树，芳草鲜美，落英缤纷，渔人甚异之。复前行，欲穷其林。
　　林尽水源，便得一山，山有小口，仿佛若有光。便舍船，从口入。初极狭，才通人。复行数十步，豁然开朗。土地平旷，屋舍俨然，有良田美池桑竹之属。阡陌交通，鸡犬相闻。其中往来种作，男女衣着，悉如外人。黄发垂髫，并怡然自乐。
　　见渔人，乃大惊，问所从来。具答之。便要还家，设酒杀鸡作食。村中闻有此人，咸来问讯。自云先世避秦时乱，率妻子邑人来此绝境，不复出焉，遂与外人间隔。问今是何世，乃不知有汉，无论魏晋。此人一一为具言所闻，皆叹惋。余人各复延至其家，皆出酒食。停数日，辞去。此中人语云：“不足为外人道也。”(间隔 一作：隔绝)
　　既出，得其船，便扶向路，处处志之。及郡下，诣太守，说如此。太守即遣人随其往，寻向所志，遂迷，不复得路。
　　南阳刘子骥，高尚士也，闻之，欣然规往。未果，寻病终，后遂无问津者。

******陋室铭******
山不在高，有仙则名。水不在深，有龙则灵。斯是陋室，惟吾德馨。苔痕上阶绿，草色入帘青。谈笑有鸿儒，往来无白丁。可以调素琴，阅金经。无丝竹之乱耳，无案牍之劳形。南阳诸葛庐，西蜀子云亭。孔子云：何陋之有？



### 实践Task： 爬取豆瓣图书Top250
方法是： Requests + Lxml
- 将结果存储为结构化的csv文件格式
- 代码： [doubanbook.py](./book_src/doubanbook.py)

### 使用API爬取数据
- 返回的一般是JSON格式的数据， 需要用JSON解析库解析JSON数据
- 可以用Requests库发起请求（GET或者POST）。但一般来说，开放的API有一些限制： 需要验证、或者有调用次数的限制、或者需要apikey才有调用权限。
- **API爬虫很常见，比如开源的聊天机器人、天气、地图等等**

In [8]:
## json
import json
jsonstring = '{"name":"xiaoming", "gender":"man"}'
json_data = json.loads(jsonstring)
print(json_data["name"])

xiaoming


In [9]:
# Python2会有中文编码问题， 打印json字符串，输出的中文就成了unicode码
import json
d = {'name': '张三', 'age': '1'}
# Python3: {'name': '张三', 'age': '1'}
# Python2: {'age': '1', 'name': '\xe5\xbc\xa0\xe4\xb8\x89'}
print(d)
# Python2: jd = json.dumps(d, ensure_ascii=False, encoding='utf-8'))
jd = json.dumps(d, ensure_ascii=False)
print(jd)

{'age': '1', 'name': '张三'}
{"age": "1", "name": "张三"}


### 数据库存储
- MongoDB数据库  
MongoDB是一种非关系型数据库(NoSQL)。NoSQL可以解决大规模数据集合多重数据种类的问题。  
NoSQL分为4大类： 键值存储数据库Redis、列存储数据库Hbase、文档型数据库MongoDB、图形数据库Graph。   
数据库和集合，类似于Excel文件和其中的表格，一个数据库可以有多个集合。  
    - 安装MongoDB数据库： sudo apt install mongodb-clients, sudo apt install mongodb-server
    - 配置数据库： mongod -dbpath path
    - 启动： mongo， 使用MongoDB期间， 启动服务的窗口不能关闭
    - Python安装pymong库： pip3 install pymongo
    - 导出数据： mongoexport -d mydb -c test --csv -f name,gender,age -o test.csv    

- MySQL
最流行的开源的关系型数据库

In [10]:
import pymongo
client = pymongo.MongoClient('localhost', 27017)  #连接数据库
mydb = client['mydb']  #新建mydb数据库
test = mydb["test"]  #新建test数据集合
# 删除一个集合中的所有文档
result = test.delete_many({})
test.insert_one({"name":"Ming", "gender":"male", "age":"30"})
## 在MongoDB设置的路径下， 打开命令行窗口，运行以下命令导出数据
# -d： 数据库， -c： 数据集合， -f： 导出的字段
# mongoexport -d mydb -c test --csv -f name,gender,age -o test.csv

### 实践Task： 爬取豆瓣音乐Top250、豆瓣电影Top250
方法是： Requests + Lxml + re + **MongoDB**
- 代码： [doubanmusic.py](./book_src/doubanmusic.py)
- 修复了一个bug， 网页代码有变化
- 导出结果： mongoexport -d mydb -c musictop --csv -f name,author,style,time,publisher,score -o douban_music.csv

### 多进程爬虫
可以使用Python的multiprocessing库的进程池方法进行多进程爬虫。

### 实践Task： 爬取58同城二手市场的商品信息
方法是： Requests + Lxml + **MongoDB + Muitiprocessing/Pool**  
- 代码： [58project/main.py](./book_src/58project/main.py)
- 可以参考的一个思想，也是在实践中常常遇到的问题： 如果由于某些原因，爬虫程序中断，如何**不手动更改却可以实现断点续爬**？  
代码中的实现方法是一个参考。

In [11]:
'''
#导入multiprocessing库的Pool模块

#创建进程池，processes参数为设置进程的个数
pool = Pool(processes=4)

# 用map()函数进行进程运行
# func参数为需运行的函数，在爬虫实战中，为爬虫函数
# iterable为迭代参数，在爬虫实战中，可为多个URL列表进行迭代。
pool.map(func,urls)
'''
from multiprocessing import Pool

### 小结
#### 爬虫原理
结合实例，了解搭建爬虫的逻辑和思路，分析和拆解问题。
#### 爬虫三大库
- Requests， GET和POST请求
- Lxml， Xpath方法
- BeautifulSoup， select方法  

#### Python re模块、json模块
另外还有： time、 csv、 try/except、 pymongo、multiprocessing、

#### 除了三大库，还有与爬虫相关的方方面面、细节 & 技巧
- 数据库MongoDB
- 多进程/多线程
- 异常判断
- 断点自动续爬
- 

#### 爬虫实例汇总
- 爬取糗事百科的段子
- 爬取《斗破苍穹》全文小说
- 爬取酷狗Top500的数据
- 爬取豆瓣图书Top250、豆瓣音乐Top250、豆瓣电影Top250
- 使用API爬取数据：百度地图、高德地图
- 爬取58同城二手市场的商品信息
- 常见方法：
    - Requests + Lxml/re + MongoDB + Muitiprocessing/Pool


In [12]:
## 与爬虫or文本处理有关的Python模块/库
import requests
import re
from lxml import etree
from bs4 import BeautifulSoup
import json
import time
import csv
import pymongo
from multiprocessing import Pool